In [4]:
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
from pyarrow import feather

from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")

In [10]:
max_iter = 500

results = []
main_url = 'https://www.indeed.com/jobs/?'

driver = webdriver.Chrome(options=chrome_options)

params = {
        'q':'Data Scientist',
        'l':'New%20York%2C%20NY',
        'sc':'0kf%3Ajt(fulltime)%3B'
    }
    
params_unfurled = '&'.join([f'{k}={params[k]}' for k in params.keys()])
new_url = main_url + params_unfurled


driver.get(new_url)
wait = WebDriverWait(driver, 10)

for i in range(max_iter):
    try:
        wait.until(EC.element_to_be_clickable((By.ID, "popover-x")))
        driver.find_element(By.ID, "popover-x").click()
    except:
        pass
        
        
    jobs = driver.find_elements(By.CLASS_NAME, "jobTitle")
    job_count = 0
    for job in jobs:
        
        try:
            wait.until(EC.element_to_be_clickable(job))
            job.click()
        except:
            print('couldnt wait to click a job')
        try:
            
            wait.until(EC.element_to_be_clickable((By.ID, "vjs-container-iframe")))
            driver.switch_to.frame("vjs-container-iframe")
            
            try:
                soup = BeautifulSoup(driver.page_source)

                result = {
                    'title' : soup.find(class_="jobsearch-JobInfoHeader-title").text,
                    'company' : soup.find(class_='jobsearch-JobInfoHeader-subtitle').text,
                    'salary_and_type' : soup.find(id='salaryInfoAndJobType').text,
                    'desc' : soup.find(id='jobDescriptionText').text.replace('\n',' ')
                }
                results.append(result)
                job_count += 1
            except:
                print('bad soup')
            
        except:
            print('couldnt click frame')
            
        
            
        driver.switch_to.default_content()
    try:
        driver.find_element(by=By.CSS_SELECTOR, value="[aria-label=Next]").click()
    except:
        print('couldnt click next')
    
    print(f'added {job_count} more jobs, {i+1} / {max_iter}')
    
driver.close()

added 15 more jobs, 1 / 500
bad soup
added 14 more jobs, 2 / 500
added 15 more jobs, 3 / 500
added 15 more jobs, 4 / 500
added 15 more jobs, 5 / 500
added 15 more jobs, 6 / 500
added 15 more jobs, 7 / 500
couldnt click frame
added 14 more jobs, 8 / 500
added 15 more jobs, 9 / 500
added 15 more jobs, 10 / 500
added 15 more jobs, 11 / 500
added 15 more jobs, 12 / 500
added 15 more jobs, 13 / 500
added 15 more jobs, 14 / 500
added 15 more jobs, 15 / 500
added 15 more jobs, 16 / 500
added 15 more jobs, 17 / 500
added 15 more jobs, 18 / 500
added 15 more jobs, 19 / 500
added 15 more jobs, 20 / 500
added 15 more jobs, 21 / 500
added 15 more jobs, 22 / 500
added 15 more jobs, 23 / 500
added 15 more jobs, 24 / 500
added 15 more jobs, 25 / 500
added 15 more jobs, 26 / 500
added 15 more jobs, 27 / 500
added 15 more jobs, 28 / 500
added 15 more jobs, 29 / 500
added 15 more jobs, 30 / 500
added 15 more jobs, 31 / 500
added 15 more jobs, 32 / 500
added 15 more jobs, 33 / 500
added 15 more jobs, 34 

MaxRetryError: HTTPConnectionPool(host='localhost', port=56466): Max retries exceeded with url: /session/2051787d04c0e256a48ec22b90d5197f/frame (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7c9ad4f490>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [11]:
df = pd.DataFrame(results)

In [12]:
df

,title,company,salary_and_type,desc
0,Data Scientist - job post,"Sia Partners20 reviewsNew York, NY 10006","$90,000 - $145,000 a year - Full-time",Company Description Sia Partners is a next...
1,Data Scientist - job post,"MurmurationNew York, NY 10005","$120,000 a year - Full-time",About Murmuration/What We Do Murmuration is...
2,Associate Data Scientist - job post,"T. Rowe Price597 reviewsNew York, NY",Full-time,There is a place for you at T. Rowe Price ...
3,Senior Data Scientist - job post,"GALE Partners15 reviewsNew York, NY",Full-time,About GALE: GALE is a creative media consultan...
4,Data Scientist - Role can be remote - job post,"MARS1,976 reviewsNewark, NJ 07102•Remote",Full-time,The Petcare Segment consists of five Division...
...,...,...,...,...
1861,Senior Data Analyst - job post,"SCO Family of Services557 reviewsGlen Cove, NY...","$60,000 - $65,000 a year - Full-time",GENERAL RESPONSIBILITIES Design and maintai...
1862,Senior Data Scientist - Delivery Insights - jo...,"Indeed1,044 reviewsUnited States•Remote","$139,000 - $201,000 a year - Full-time",Your Job Delivery Insights provides dev...
1863,"Director, Data Science - job post","Indeed1,044 reviewsUnited States•Remote","$253,000 - $367,000 a year - Full-time",Your Job We’re looking for a senior lea...
1864,"Director, Statistical Programming - job post",Everest Clinical Research11 reviewsUnited Stat...,Full-time,Everest Clinical Research (“Everest”) is a f...


In [13]:
feather.write_feather(df, 'results_ny.feather')